In [1]:
import lib.datautility as du
import numpy as np
import lib.tf_network2 as tfnet
import lib.evaluationutility as eu
from lib.tf_network2 import Network, Cost, Normalization, Optimizer
import tensorflow as tf
import session_info

In [2]:
import zipfile
with zipfile.ZipFile('resources/resources.zip','r') as zf:
    zf.extractall('resources/')

In [3]:
data, headers = du.read_csv('resources/remnant.csv', max_rows=100)
du.print_descriptives(data, headers)


data, headers = du.read_csv('resources/experimental.csv', max_rows=100)
du.print_descriptives(data, headers)

-- loading resources/remnant.csv...(100%)   
DESCRIPTIVES     100 Rows, 27 Columns   
0 sequence_id_t... (int):      0 missing (0.0%)     mean=5969.00 (SD=0.00)                                                                           
1 student_user_... (int):      0 missing (0.0%)     mean=73142.34 (SD=3621.97)                                                                       
2 target_assign... (int):      0 missing (0.0%)     mean=216684.95 (SD=0.05)                                                                         
3 target_assign... (int):      0 missing (0.0%)     mean=0.59 (SD=0.49)                                                                              
4 prior_assignm... (int):      0 missing (0.0%)     mean=216679.72 (SD=6.26)                                                                         
5 prior_assignm... (string):   0 missing (0.0%)     100 unique values                                                                                
6 prior_assign

In [4]:
key = [1,2,0]
iden = [4]
cov = [10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26]
lab = [[3]]
order = 7

seq = tfnet.format_sequence_from_file('resources/remnant.csv',
                                          key,lab,cov,iden,order)
print('formatting feature columns...')
for i in range(len(seq['x'])):
    for j in range(len(seq['x'][i])):
        try:
            seq['x'][i][j] = np.array(seq['x'][i][j], dtype=np.float32)
        except TypeError:
            print('error...')
            print(seq['x'][i])
            exit(1)
seq['x'] = tfnet.sequence_impute_missing(seq['x'])

print('formatting output labels...')
seq['y'] = tfnet.use_last_multi_label(seq['y'], 0)

print('done!')

-- loading resources/remnant.csv...(100%)   
-- preparing to format sequence from file...
-- formatting sequence...(0.4%)  

C:\Users\asalesl\Google Drive\iesJohannReloop\reloopPaper\Python Code\Study_2\lib\tf_network2.py:2001: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  seq['x'] = np.array(x)
C:\Users\asalesl\Google Drive\iesJohannReloop\reloopPaper\Python Code\Study_2\lib\tf_network2.py:2003: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  seq['iden'] = np.array(id)


-- formatting sequence...(100%)   
formatting feature columns...


C:\Users\asalesl\anaconda3\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


formatting output labels...
done!


In [5]:
key = [1,2,0]
iden = [4]
cov = [10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26]
lab = [[3]]
order = 7

seqex = tfnet.format_sequence_from_file('resources/experimental.csv',
                                          key,lab,cov,iden,order)
print('formatting feature columns...')
for i in range(len(seqex['x'])):
    for j in range(len(seqex['x'][i])):
        try:
            seqex['x'][i][j] = np.array(seqex['x'][i][j], dtype=np.float32)
        except TypeError:
            print('error...')
            print(seqex['x'][i])
            exit(1)
seqex['x'] = tfnet.sequence_impute_missing(seqex['x'])

print('formatting output labels...')
seqex['y'] = tfnet.use_last_multi_label(seqex['y'], 0)

print('done!')

-- loading resources/experimental.csv...(100%)   
-- preparing to format sequence from file...
-- formatting sequence...(100%)   
formatting feature columns...
formatting output labels...
done!


In [6]:
n_cov = len(seq['x'][0][0])

max_epochs = 200
use_validation = True
hidden = [50]
batch = [64]
layers = [1]
keep = [.5]
step = [1e-3]
perf = []
threshold = [0.0001]
optimizer = [Optimizer.ADAM]

tf.compat.v1.disable_eager_execution()

tfnet.describe_multi_label(seq['y'], True)

======  Label Description  =============
-- Number of Samples: 129835
-- Number of Label Sets: 1
---- 1: 1 Label



{'n_samples': 129835, 'n_label_sets': 1, 'n_labels': [1]}

In [7]:
tf.compat.v1.reset_default_graph()
tf.compat.v1.set_random_seed(1)
np.random.seed(1)

net = Network('study2_model').add_input_layer(n_cov, normalization=Normalization.Z_SCORE)
net.add_lstm_layer(hidden[0], peepholes=True)

net.begin_multi_output(cost_methods=[Cost.BINARY_CROSS_ENTROPY])
net.add_dropout_layer(1, keep=keep[0], activation=tf.nn.sigmoid)
net.end_multi_output()

net.set_default_cost_method(Cost.CROSS_ENTROPY)
net.set_optimizer(optimizer[0])

net.train(x=seq['x'],
          y=seq['y'],
          step=step[0],
          use_validation=True,
          max_epochs=max_epochs, threshold=threshold[0], batch=batch[0])

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
WARNING - Incomplete validation set information was provided. Using default 70/30% split.
            Training Network            
-3 layers: 17n -> 50n -> 1n
-200 epochs
-step size = 0.001
-batch size = 64
Epoch        Cost    Val Cost   Delta         Time


C:\Users\asalesl\Google Drive\iesJohannReloop\reloopPaper\Python Code\Study_2\lib\tf_network2.py:1048: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  np.array([a[i] for a in series_label])]).reshape((len(series_batch), n_timestep, -1)))


Epoch 1     0.3919    0.2640    0.0000      162.4s
Epoch 2     0.3695    0.2648   -0.0004      152.2s
Epoch 3     0.3670    0.2635    0.0003      155.2s
Epoch 4     0.3649    0.2615    0.0006      159.8s
Epoch 5     0.3641    0.2608    0.0005      153.1s
Epoch 6     0.3629    0.2609    0.0006      153.8s
Epoch 7     0.3624    0.2612    0.0007      160.2s
Epoch 8     0.3613    0.2607    0.0006      170.3s
Epoch 9     0.3616    0.2611    0.0001      152.1s
Total Time: 1419.1s


In [8]:
net.save_model_weights()      # comment to use a previously-saved model
#
# net.build()                 # uncomment to use previously-saved model
# net.restore_model_weights() # uncomment to use previously-saved model

In [9]:
print('writing predictions to file...')
du.write_csv(tfnet.flatten_sequence(np.array([np.array([t[-1]]) for t in net.predict(seqex['x'])[0]]),
                                        seqex['key'].reshape((-1,3))), 'model_predictions.csv',
             ['user_id','target_assignment_id','target_sequence_id','pcomplete'])
print('done!')

writing predictions to file...


C:\Users\asalesl\Google Drive\iesJohannReloop\reloopPaper\Python Code\Study_2\lib\tf_network2.py:1428: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  pred.append(np.array(out_p))


done!


In [10]:
import graph_def_editor 
import warnings
import time
import csv
import sys
import os
import pandas 
from sklearn.metrics import f1_score, r2_score
from sklearn.metrics import mean_squared_error
from skll.metrics import kappa as kpa
import pickle
import psycopg2 as pg
import functools
import Levenshtein as lev
from nltk.tokenize import word_tokenize

session_info.show()

In [11]:
net.session.close()